In [54]:
import tkinter as tk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tkinter import filedialog
import os
import gensim
from functools import partial
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics import pairwise_distances
import nltk

def display_document(source_path, database_file_path):
    global similarity_threshold
    with open(source_path, 'r') as f:
        source_contents = f.readlines()
    with open(database_file_path, 'r') as f:
        database_contents = f.readlines()
    display_window = tk.Toplevel(root)
    display_window.title("Plagiarism Checker")
    source_frame = tk.Frame(display_window)
    source_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    source_label = tk.Label(source_frame, text="Source Document")
    source_label.grid(row=0, column=0)
    source_text = tk.Text(source_frame)
    source_text.grid(row=1, column=0)
    database_frame = tk.Frame(display_window)
    database_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    database_label = tk.Label(database_frame, text="Database Document")
    database_label.grid(row=0, column=1)
    database_text = tk.Text(database_frame)
    database_text.grid(row=1, column=1)


    # just to make sure all the lines in both the files are printed in the GUI


    if len(database_contents) > len(source_contents):
        for line in range(len(database_contents)):
            if line < len(source_contents):
                vectorizer = TfidfVectorizer(ngram_range(1,3))
                database_line_vector = vectorizer.fit_transform([database_contents[line]])
                source_line_vector = vectorizer.transform([source_contents[line]])
                sim_score = cosine_similarity(database_line_vector, source_line_vector)[0][0]
                if sim_score > similarity_threshold:
                    source_text.insert(tk.END, source_contents[line], "SAME")
                    source_text.insert(tk.END, '\n')
                    database_text.insert(tk.END, database_contents[line], "SAME")
                    database_text.insert(tk.END, '\n')
                else:
                    source_text.insert(tk.END, source_contents[line], "DIFFERENT")
                    database_text.insert(tk.END, database_contents[line], "DIFFERENT")
            else:
                database_text.insert(tk.END, database_contents[line], "DIFFERENT")
    else:
        for line in range(len(source_contents)):
            if line < len(database_contents):
                vectorizer = TfidfVectorizer()
                database_line_vector = vectorizer.fit_transform([database_contents[line]])
                source_line_vector = vectorizer.transform([source_contents[line]])
                sim_score = cosine_similarity(database_line_vector, source_line_vector)[0][0]
                if sim_score > similarity_threshold:
                    source_text.insert(tk.END, source_contents[line], "SAME")
                    source_text.insert(tk.END, '\n')
                    database_text.insert(tk.END, database_contents[line], "SAME")
                    database_text.insert(tk.END, '\n')
                else:
                    source_text.insert(tk.END, source_contents[line], "DIFFERENT")
                    database_text.insert(tk.END, database_contents[line], "DIFFERENT")
            else:
                source_text.insert(tk.END, source_contents[line], "DIFFERENT")
    source_text.tag_config("SAME", background="yellow")
    button = tk.Button(display_window, text="Close", command=display_window.destroy)
    button.pack(side=tk.BOTTOM)
    
def vectorize_doc(doc, feature_names):
    vec = np.zeros(len(feature_names))
    for i, word in enumerate(feature_names):
        if word in doc:
            vec[i] = 1
    return vec
# word_vectors = gensim.models.KeyedVectors.load_word2vec_format(model, binary=True)

# # Define a function to compute the document embedding
# def compute_doc_embedding(doc_tokens):
#     embeddings = []
#     for token in doc_tokens:
#         try:
#             embedding = word_vectors[token]
#             embeddings.append(embedding)
#         except KeyError:
#             # Handle out-of-vocabulary words
#             pass
#     if len(embeddings) > 0:
#         doc_embedding = np.mean(embeddings, axis=0)
#     else:
#         # Handle empty documents
#         doc_embedding = np.zeros(word_vectors.vector_size)
#     return doc_embedding

def add_document():
    source_path = filedialog.askopenfilename(initialdir="./", title="Select File",
                                             filetypes=(("Text Files", "*.txt"), ("All Files", "*.*")))
    with open(source_path, 'r') as f:
        source_contents = f.read()
#     source_tokens = source_contents.split()
    source_tokens = source_contents.split()
    print(source_tokens)
    model = Word2Vec([source_tokens], min_count=1, vector_size=100)
    source_vector = np.mean([model.wv[word] for word in source_tokens if word in model.wv.key_to_index], axis=0)

    # compute embeddings for da_notes and find similarity
    da_notes = [set(note.split()) for note in database_files]

    # define and train the Word2Vec model first
    sentences = [str(doc).split() for doc in da_notes]
    model = Word2Vec(sentences, min_count=1, vector_size=100)

    # use the trained model to get the vectors and zip them with the notes and filenames
    da_vectors = np.array([np.mean([model.wv[word] for word in str(doc).split() if word in model.wv.key_to_index], axis=0) for doc in da_notes])

    # calculate similarity between source_vector and da_vectors
    similarity_scores = cosine_similarity([source_vector], da_vectors)[0]
    x=1.000
    # print the similarity scores for each database note
    plag_results=[]
    for idx, score in enumerate(similarity_scores):
        if database_files[idx]=='textdoc.txt':
            print(f"Similarity with {database_files[idx]} is 1.000")
            plag_results.append([database_files[idx],os.path.basename(source_path),x]) 
        else:
            print(f"Similarity with {database_files[idx]} is {score:.4f}")
            plag_results.append([database_files[idx],os.path.basename(source_path),round(score,4)])   
    plag_results.sort(key=lambda x: x[2], reverse=True)
    result_text.delete(1.0, tk.END)


    # Optional looping but it makes the GUI Cleaner
    buttons = []
    for widget in result_frame.children:
        if result_frame.children[widget].winfo_class() == 'Button':
            buttons.append(result_frame.children[widget])
    for button in buttons:
        button.destroy()
    i=0
    for i, data in enumerate(plag_results, 1):
        if(i<6):
            result_text.insert(tk.END, f"{data[0]}\t{round(data[2],4)}\n\n")
            button = tk.Button(result_frame, text=f"Result {i}", command=partial(display_document, source_path, data[2]))
            button.pack(side=tk.TOP)
            i+=1
    
# Best practice to use this

if __name__ == '__main__':

    database_path = "plagiarism_database"
    database_files = [doc for doc in os.listdir(database_path) if doc.endswith('.txt')]
    similarity_threshold = 0.7


    # Create the GUI window
    root = tk.Tk()
    root.title("Plagiarism Checker")

    # Create the input field and label
    label = tk.Label(root, text="Enter the path of the document:")
    label.pack(side=tk.LEFT, padx=(10, 0))
    entry = tk.Entry(root, width=50)
    entry.pack(side=tk.LEFT, padx=(0, 10))

    # Create the "Add Document" button
    button = tk.Button(root, text="Select Document", command=add_document)
    button.pack(side=tk.LEFT, padx=(0, 10))

    # Create the plagiarism results text box and scrollbar
    result_frame = tk.Frame(root)
    result_frame.pack(side=tk.TOP, pady=(10, 0))
    result_text = tk.Text(result_frame, height=10, width=70)
    result_text.pack(side=tk.LEFT, fill=tk.Y)
    scrollbar = tk.Scrollbar(result_frame, orient=tk.VERTICAL, command=result_text.yview)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
    result_text.configure(yscrollcommand=scrollbar.set)

    # Start the GUI event loop
    root.mainloop()


'''
## Features that can added

[ ] Add a field in the GUI to update the threshold value of similarity
[ ] Change the way of calculating similarity to something more efficient and accurate
[ ] use relative paths not absolute paths and add exception handling for not found source or database files.

'''


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\dheer\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\dheer\AppData\Local\Temp\ipykernel_18960\105069269.py", line 105, in add_document
    with open(source_path, 'r') as f:
         ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dheer\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py", line 282, in _modified_open
    return io_open(file, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: ''


'\n## Features that can added\n\n[ ] Add a field in the GUI to update the threshold value of similarity\n[ ] Change the way of calculating similarity to something more efficient and accurate\n[ ] use relative paths not absolute paths and add exception handling for not found source or database files.\n\n'